# init

In [1]:
import numpy as np
from pprint import pprint
from copy import deepcopy
import sys
from pathlib import Path


from laboneq.contrib.example_helpers.generate_descriptor import generate_descriptor
#from laboneq.contrib.example_helpers.generate_device_setup import generate_device_setup
from laboneq.dsl.device import DeviceSetup
from laboneq.simple import *
from laboneq.dsl.device import DeviceSetup
from laboneq.dsl.calibration import Oscillator, SignalCalibration
from laboneq.dsl.enums import ModulationType




# descriptor = generate_descriptor(
#     #pqsc=[""], # 장비 여러개 사용시
#     shfqc_6=["DEV12256"], 
#     number_data_qubits=3,
#     multiplex=True,
#     number_multiplex=6,
#     include_cr_lines=False,
#     include_ef_lines=True,
#     get_zsync=False,  # Only set to True when using real device
#     save = True,
#     filename="1port",
#     ip_address="192.168.0.83"
# )


#descriptor
#setup = DeviceSetup.from_descriptor(yaml_text=descriptor, server_host="192.168.0.83")
descriptor_candidates = [
    Path("examples/selectiveRIP/Descriptors/1port.yaml"),
    Path("Descriptors/1port.yaml"),
]
descriptor_path = next((p for p in descriptor_candidates if p.exists()), descriptor_candidates[0])
setup = DeviceSetup.from_yaml(filepath=str(descriptor_path.resolve()), server_host="192.168.0.83")
#setup
setup.instruments[0].device_options = 'SHFQC/PLUS/QC6CH'
#setup.instruments


bus_ids = [f"b{i}" for i in range(3)]
bus_port = [4,5,6] #used 1,2,3 for qubit drive

for i, bus in zip(bus_port,bus_ids):
    setup.add_connections(
        setup.instruments[0].uid,
        # each bus uses its own drive:
        create_connection(
            to_signal=f"{bus}/drive",
            ports=f"SGCHANNELS/{i}/OUTPUT"
        ))

# Calibrate qubit drive/measure lines for oscillator phase increments
qubit_ids = [uid for uid in setup.logical_signal_groups if uid.startswith("q")]
for qubit in qubit_ids:
    for line, frequency, mod_type in [
        ("drive", 5e9, ModulationType.HARDWARE),
        ("drive_ef", 6e9, ModulationType.HARDWARE),
        ("measure", 4e9, ModulationType.SOFTWARE),
    ]:
        logical_signal = setup.logical_signal_by_uid(f"{qubit}/{line}")
        oscillator = Oscillator(modulation_type=mod_type)
        logical_signal.calibration = SignalCalibration(
            local_oscillator=Oscillator(frequency=frequency),
            oscillator=oscillator,
        )
        if line == "measure":
            acquire_signal = setup.logical_signal_by_uid(f"{qubit}/acquire")
            acquire_signal.calibration = SignalCalibration(
                local_oscillator=Oscillator(frequency=frequency),
                oscillator=oscillator,
            )

project_root = next(
    (p for p in [Path.cwd(), *Path.cwd().parents] if (p / "qpu_types").exists() and (p / "helper.py").exists()),
    Path.cwd(),
)
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

from qpu_types.fixed_transmon import FixedTransmonQubit, FixedTransmonQubitParameters
from qpu_types.bus_cavity import BusCavity, BusCavityParameters
from qpu_types.fixed_transmon.operations import FixedTransmonOperations
from qpu_types.bus_cavity.operations import BusCavityOperations
from laboneq.dsl.quantum.qpu import QPU, QuantumPlatform
from helper import load_qubit_parameters, save_qubit_parameters

qubit_uids = [uid for uid in setup.logical_signal_groups if uid.startswith("q")]
bus_uids = [uid for uid in setup.logical_signal_groups if uid.startswith("b")]

qubits = FixedTransmonQubit.from_device_setup(
    setup, qubit_uids=qubit_uids)
buses = BusCavity.from_device_setup(
    setup, qubit_uids=bus_uids)

qpu = QPU(quantum_elements={"qubits" : qubits, "bus" : buses}, quantum_operations=[FixedTransmonOperations, BusCavityOperations])


# from laboneq.simple import workflow
# folder_store = workflow.logbook.FolderStore("./experiment_store") 
# folder_store.activate()
# #folder_store.deactivate()
# #workflow.logbook.LoggingStore().activate()
# #workflow.logbook.LogbookStore().deactivate()

import os
from datetime import datetime
def find_latest_json(folder_path):
    files = [f for f in os.listdir(folder_path)]
    timestamps = []
    for file in files:
        try:
            # Extract a timestamp assuming it's included in the filename
            timestamp_str = file.split('_', 1)[0]  # Assuming YYYYMMDDHHMMSS format
            timestamp = datetime.strptime(timestamp_str, '%Y%m%d-%H%M%S')
            timestamps.append((timestamp, file))
        except ValueError:
            print("fuck")
            continue  # Skip files that do not match the timestamp format

       # Find the most recent file
    if timestamps:
        latest_file = max(timestamps, key=lambda x: x[0])[1]
        return os.path.join(folder_path, latest_file)
    return None

qb_pars_file = find_latest_json("./qpu_parameters/")
print(f"LOADED: {qb_pars_file}")

from qpu_types.fixed_transmon.operations import FixedTransmonOperations
from qpu_types.bus_cavity.operations import BusCavityOperations
import laboneq.dsl.quantum.qpu as qpu_mod

class CombinedOperations(FixedTransmonOperations, BusCavityOperations):
    pass

qpu_mod.CombinedOperations = CombinedOperations

qpu = load(qb_pars_file)

buses = qpu.groups.bus
qubits = qpu.groups.qubits


from laboneq.simple import workflow
folder_store = workflow.logbook.FolderStore("./experiment_store")
folder_store.activate()

fuck
LOADED: ./qpu_parameters/20260221-0955_readout_opt


In [2]:
from laboneq.simple import Session
session = Session(setup)
session.connect(ignore_version_mismatch=False, do_emulation=False)
#session.disconnect()

[2026.02.21 09:55:53.416] INFO    Logging initialized from [Default inline config in laboneq.laboneq_logging] logdir is /Users/yalgaeahn/Research/code/qubit-experiment/examples/selectiveRIP/laboneq_output/log
[2026.02.21 09:55:53.419] INFO    VERSION: laboneq 25.10.3
[2026.02.21 09:55:53.420] INFO    Connecting to data server at 192.168.0.83:8004
[2026.02.21 09:55:53.473] INFO    Connected to Zurich Instruments LabOne Data Server version 25.10.1.4 at 192.168.0.83:8004
[2026.02.21 09:55:53.610] INFO    Configuring the device setup
[2026.02.21 09:55:53.833] INFO    The device setup is configured


# Multiplexed IQ cloud

In [9]:
qubits[0].readout_integration_parameters()
qubits[0].readout_parameters()

('measure',
 {'amplitude': 0.522,
  'length': 8.192000000000001e-07,
  'pulse': {'function': 'GaussianSquare',
   'risefall_sigma_ratio': 3.0,
   'sigma': 0.03}})

In [7]:
from experiments import iq_cloud_common, iq_cloud
q = qubits[0]
qq = qubits[1]

temporary_parameters = {}
q_temp_pars = deepcopy(q.parameters)
qq_temp_pars = deepcopy(qq.parameters)
qq_temp_pars.readout_integration_delay = 250e-9
#q_temp_pars.readout_integration_delay = 250e-9
temporary_parameters[q.uid] = q_temp_pars
temporary_parameters[qq.uid] = qq_temp_pars
#######################################################################

options = iq_cloud.experiment_workflow.options()
options.do_analysis(True)
options.update(False)
#options.count(1024)

iq_result = iq_cloud.experiment_workflow(
    session=session,
    qpu=qpu,
    qubits=[q,qq],
    temporary_parameters=temporary_parameters,
    options=options,
).run()




[2026.02.21 09:58:46.790] INFO     ────────────────────────────────────────────────────────────────────────────── 
[2026.02.21 09:58:46.791] INFO      Workflow 'iq_cloud': execution started at 2026-02-21 00:58:46.789703Z         
[2026.02.21 09:58:46.791] INFO     ────────────────────────────────────────────────────────────────────────────── 
[2026.02.21 09:58:46.803] INFO    Task 'temporary_qpu': started at 2026-02-21 00:58:46.803374Z
[2026.02.21 09:58:46.804] INFO    Task 'temporary_qpu': ended at 2026-02-21 00:58:46.804764Z
[2026.02.21 09:58:46.808] INFO    Task 'temporary_quantum_elements_from_qpu': started at 2026-02-21 
[2026.02.21 09:58:46.809] INFO    00:58:46.807989Z
[2026.02.21 09:58:46.811] INFO    Task 'temporary_quantum_elements_from_qpu': ended at 2026-02-21 00:58:46.811534Z
[2026.02.21 09:58:46.818] INFO    Task 'create_experiment': started at 2026-02-21 00:58:46.817354Z
[2026.02.21 09:58:46.837] INFO    Task 'create_experiment': ended at 2026-02-21 00:58:46.837635Z
[202

[2026.02.21 09:58:46.841] ERROR   Task 'compile_experiment': failed at 2026-02-21 00:58:46.841119Z with: 


[2026.02.21 09:58:46.841] ERROR   LabOneQException("The experiment signals q0/acquire, q1/acquire all touch 


[2026.02.21 09:58:46.841] ERROR   LabOneQException("The experiment signals q0/acquire, q1/acquire all touch 


[2026.02.21 09:58:46.842] ERROR   physical channel 'SHFQC_DEV12256/qachannels_0_input', but provide conflicting 


[2026.02.21 09:58:46.842] ERROR   physical channel 'SHFQC_DEV12256/qachannels_0_input', but provide conflicting 


[2026.02.21 09:58:46.842] ERROR   settings for calibration field 'port_delay'.")


[2026.02.21 09:58:46.842] ERROR   settings for calibration field 'port_delay'.")


[2026.02.21 09:58:46.844] INFO    Task 'compile_experiment': ended at 2026-02-21 00:58:46.843902Z
[2026.02.21 09:58:46.845] INFO     ────────────────────────────────────────────────────────────────────────────── 
[2026.02.21 09:58:46.846] INFO      Workflow 'iq_cloud': execution ended at 2026-02-21 00:58:46.845166Z           
[2026.02.21 09:58:46.847] INFO     ────────────────────────────────────────────────────────────────────────────── 


LabOneQException: The experiment signals q0/acquire, q1/acquire all touch physical channel 'SHFQC_DEV12256/qachannels_0_input', but provide conflicting settings for calibration field 'port_delay'.

## INTEGRATION Pipeline Validation (New)

아래 셀은 **DISCRIMINATION 경로 없이** `INTEGRATION + SINGLE_SHOT` 기반으로\n
`two_qubit_readout_calibration` -> `two_qubit_state_tomography` 분석이 동작하는지 검증합니다.


In [ ]:
print(qubits[0].parameters.readout_resonator_frequency)
print(qubits[1].parameters.readout_resonator_frequency)

In [ ]:
from experiments import two_qubit_readout_calibration
from experiments import two_qubit_state_tomography
import numpy as np

# Reuse existing variables from notebook: session, qpu, ctrl, targ, bus, temporary_parameters
if "ctrl" not in globals() or "targ" not in globals():
    ctrl = qubits[1]
    targ = qubits[0]
if "bus" not in globals():
    bus = buses[0]
if "temporary_parameters" not in globals():
    temporary_parameters = {}

bus = buses[0]
def unwrap_output(obj):
    cur = obj
    for _ in range(24):
        if hasattr(cur, "output"):
            cur = cur.output
            continue
        return cur
    return cur


def _is_reference_like(obj):
    return obj is not None and obj.__class__.__name__ == "Reference"


def _contains_reference(obj, depth=0, max_depth=12):
    if depth > max_depth:
        return False
    cur = unwrap_output(obj)
    if _is_reference_like(cur):
        return True
    if isinstance(cur, dict):
        return any(_contains_reference(v, depth + 1, max_depth) for v in cur.values())
    if isinstance(cur, (list, tuple)):
        return any(_contains_reference(v, depth + 1, max_depth) for v in cur)
    return False


def _iter_tasks(node):
    tasks = getattr(node, "tasks", None)
    if tasks is None:
        return []
    try:
        return list(tasks)
    except Exception:
        return []


def _task_output(tasks, key):
    try:
        out = unwrap_output(tasks[key].output)
    except Exception:
        return None
    if _is_reference_like(out):
        return None
    return out


def _assemble_from_analysis_tasks(analysis_node):
    tasks = getattr(analysis_node, "tasks", None)
    if tasks is None:
        return None

    assignment = _task_output(tasks, "extract_assignment_matrix")
    tomography_counts = _task_output(tasks, "collect_tomography_counts")
    mle = _task_output(tasks, "maximum_likelihood_reconstruct")
    state_metrics = _task_output(tasks, "calculate_state_metrics")
    discriminator = _task_output(tasks, "fit_discriminator_from_readout_calibration")

    if not all(isinstance(x, dict) for x in [assignment, tomography_counts, mle, state_metrics, discriminator]):
        return None

    assembled = {
        "assignment_matrix": assignment.get("assignment_matrix"),
        "assignment_counts": assignment.get("counts_matrix_soft"),
        "assignment_counts_soft": assignment.get("counts_matrix_soft"),
        "assignment_counts_hard": assignment.get("counts_matrix_hard"),
        "tomography_counts": tomography_counts.get("counts"),
        "tomography_counts_hard": tomography_counts.get("counts_hard"),
        "setting_labels": tomography_counts.get("setting_labels"),
        "shots_per_setting": tomography_counts.get("shots_per_setting"),
        "rho_hat_real": mle.get("rho_hat_real"),
        "rho_hat_imag": mle.get("rho_hat_imag"),
        "predicted_probabilities": mle.get("predicted_probabilities"),
        "predicted_counts": mle.get("predicted_counts"),
        "optimizer_success": mle.get("optimizer_success"),
        "optimizer_message": mle.get("optimizer_message"),
        "negative_log_likelihood": mle.get("negative_log_likelihood"),
        "metrics": state_metrics,
        "discriminator_model": discriminator.get("model"),
        "classification_diagnostics": discriminator.get("diagnostics"),
        "bitflip_ctrl": False,
        "bitflip_targ": False,
    }
    return assembled


def _find_analysis_node(root, depth=0, max_depth=12):
    if root is None or depth > max_depth:
        return None
    tasks = _iter_tasks(root)
    if tasks:
        names = {getattr(t, "name", "") for t in tasks}
        if "maximum_likelihood_reconstruct" in names and "extract_assignment_matrix" in names:
            return root
        for t in tasks:
            found = _find_analysis_node(t, depth + 1, max_depth)
            if found is not None:
                return found
            found = _find_analysis_node(getattr(t, "output", None), depth + 1, max_depth)
            if found is not None:
                return found
    out = getattr(root, "output", None)
    if out is not None and out is not root:
        return _find_analysis_node(out, depth + 1, max_depth)
    return None


def extract_analysis_output(workflow_result):
    analysis_node = _find_analysis_node(workflow_result)
    if analysis_node is None:
        raise RuntimeError("Could not locate analysis workflow node.")

    out = unwrap_output(getattr(analysis_node, "output", None))
    if isinstance(out, dict) and not _contains_reference(out):
        return out

    assembled = _assemble_from_analysis_tasks(analysis_node)
    if isinstance(assembled, dict):
        return assembled

    raise RuntimeError("Could not materialize concrete analysis output from workflow tasks.")


# 1) Readout calibration acquisition (INTEGRATION + SINGLE_SHOT fixed by module)
readout_cal_result_new = two_qubit_readout_calibration.experiment_workflow(
    session=session,
    qpu=qpu,
    ctrl=ctrl,
    targ=targ,
    temporary_parameters=temporary_parameters,
).run()

# 2) 2Q tomography with external readout calibration result
tomo_opts_new = two_qubit_state_tomography.experiment_workflow.options()
tomo_opts_new.do_analysis(True)
tomo_opts_new.do_readout_calibration(False)

bus_frequency = (
    bus.parameters.resonance_frequency_bus + (bus.parameters.rip_detuning or 0.0)
    if bus.parameters.resonance_frequency_bus is not None
    else 6.5e9
)
rip_amplitude = float(getattr(bus.parameters, "rip_amplitude", 0.0) or 0.0)
rip_length = float(getattr(bus.parameters, "rip_length", 64e-9) or 64e-9)

twoq_qst_result_new = two_qubit_state_tomography.experiment_workflow(
    session=session,
    qpu=qpu,
    ctrl=ctrl,
    targ=targ,
    bus=bus,
    bus_frequency=bus_frequency,
    rip_amplitude=rip_amplitude,
    rip_length=rip_length,
    rip_phase=np.pi / 2,
    readout_calibration_result=readout_cal_result_new,
    target_state="++",
    options=tomo_opts_new,
    temporary_parameters=temporary_parameters,
).run()

out_new = unwrap_output(twoq_qst_result_new.output)
analysis_new = extract_analysis_output(twoq_qst_result_new)

print("top-level task names:", [t.name for t in twoq_qst_result_new.tasks])
print("workflow output type:", type(out_new).__name__)
print("analysis keys:", list(analysis_new.keys()))
print("fidelity_to_target:", analysis_new["metrics"].get("fidelity_to_target"))
print("classifier diagnostics:", analysis_new.get("classification_diagnostics", {}))



In [ ]:
import matplotlib.pyplot as plt
import numpy as np

A_soft = np.asarray(analysis_new["assignment_matrix"], dtype=float)
C_soft = np.asarray(analysis_new["assignment_counts_soft"], dtype=float)
C_hard = np.asarray(analysis_new["assignment_counts_hard"], dtype=float)

fig, axes = plt.subplots(1, 3, figsize=(15, 4.2))

im0 = axes[0].imshow(A_soft, vmin=0.0, vmax=1.0, cmap="Blues")
axes[0].set_title("Assignment matrix (soft)")
axes[0].set_xlabel("Prepared k")
axes[0].set_ylabel("Measured i")
axes[0].set_xticks(range(4), ["00", "01", "10", "11"])
axes[0].set_yticks(range(4), ["00", "01", "10", "11"])
fig.colorbar(im0, ax=axes[0], fraction=0.046)

im1 = axes[1].imshow(C_soft, cmap="viridis")
axes[1].set_title("Calibration counts (soft)")
axes[1].set_xlabel("Prepared k")
axes[1].set_ylabel("Measured i")
axes[1].set_xticks(range(4), ["00", "01", "10", "11"])
axes[1].set_yticks(range(4), ["00", "01", "10", "11"])
fig.colorbar(im1, ax=axes[1], fraction=0.046)

im2 = axes[2].imshow(C_hard, cmap="viridis")
axes[2].set_title("Calibration counts (hard, argmax)")
axes[2].set_xlabel("Prepared k")
axes[2].set_ylabel("Measured i")
axes[2].set_xticks(range(4), ["00", "01", "10", "11"])
axes[2].set_yticks(range(4), ["00", "01", "10", "11"])
fig.colorbar(im2, ax=axes[2], fraction=0.046)

fig.tight_layout()
plt.show()


## QST Validation Suite (Pass/Fail)

아래 셀은 `analysis_new`(또는 workflow 결과)로부터 2Q QST가 정상 동작하는지 자동 검증합니다.
- Assignment matrix 정규화/대각 우세
- Tomography counts 무결성
- 재구성 density matrix 물리성(Hermitian/trace/PSD)
- target fidelity 범위


In [ ]:
import numpy as np

def _fmt_bool(x):
    return 'PASS' if bool(x) else 'FAIL'

def validate_twoq_qst_analysis(analysis_dict, *, tol=1e-6):
    A = np.asarray(analysis_dict['assignment_matrix'], dtype=float)
    C_soft = np.asarray(analysis_dict['assignment_counts_soft'], dtype=float)
    C_hard = np.asarray(analysis_dict['assignment_counts_hard'], dtype=float)
    T_soft = np.asarray(analysis_dict['tomography_counts'], dtype=float)
    T_hard = np.asarray(analysis_dict['tomography_counts_hard'], dtype=float)
    shots = np.asarray(analysis_dict['shots_per_setting'], dtype=float)
    setting_labels = list(analysis_dict['setting_labels'])

    rho = np.asarray(analysis_dict['rho_hat_real'], dtype=float) + 1j*np.asarray(analysis_dict['rho_hat_imag'], dtype=float)

    checks = {}

    checks['assignment_shape_4x4'] = (A.shape == (4, 4))
    row_sums = A.sum(axis=1) if A.shape == (4, 4) else np.array([np.nan])
    checks['assignment_rows_sum_to_1'] = np.all(np.isfinite(row_sums)) and np.allclose(row_sums, 1.0, atol=5e-3)
    checks['assignment_diagonal_better_than_random'] = (np.trace(A)/4.0) > 0.25 if A.shape == (4, 4) else False

    checks['counts_nonnegative_finite'] = (
        np.all(np.isfinite(C_soft)) and np.all(np.isfinite(C_hard)) and
        np.all(np.isfinite(T_soft)) and np.all(np.isfinite(T_hard)) and
        np.all(C_soft >= -tol) and np.all(C_hard >= -tol) and
        np.all(T_soft >= -tol) and np.all(T_hard >= -tol)
    )

    checks['tomography_row_sum_matches_shots'] = (
        T_soft.shape[0] == shots.shape[0] and np.allclose(T_soft.sum(axis=1), shots, atol=max(1e-6, 5e-3*np.max(shots)))
    )
    checks['settings_count_consistent'] = (T_soft.shape[0] == len(setting_labels))

    checks['rho_shape_4x4'] = (rho.shape == (4, 4))
    herm_err = np.linalg.norm(rho - rho.conj().T) if rho.shape == (4, 4) else np.inf
    checks['rho_hermitian'] = herm_err < 1e-8
    tr = np.trace(rho) if rho.shape == (4, 4) else np.nan
    checks['rho_trace_1'] = np.isfinite(np.real(tr)) and abs(np.real(tr) - 1.0) < 1e-6
    min_eig = float(np.min(np.linalg.eigvalsh((rho + rho.conj().T)/2.0)).real) if rho.shape == (4, 4) else -np.inf
    checks['rho_psd'] = min_eig > -1e-6

    fidelity = analysis_dict.get('metrics', {}).get('fidelity_to_target', None)
    checks['fidelity_in_range_or_none'] = (fidelity is None) or (0.0 <= float(fidelity) <= 1.0)

    summary = {
        'assignment_avg_diag': float(np.trace(A)/4.0),
        'assignment_row_sums': row_sums.tolist(),
        'hermitian_error': float(herm_err),
        'rho_trace_real': float(np.real(tr)),
        'rho_min_eigenvalue': float(min_eig),
        'fidelity_to_target': None if fidelity is None else float(fidelity),
        'num_settings': int(T_soft.shape[0]),
        'shots_minmax': (float(np.min(shots)), float(np.max(shots))) if shots.size else (np.nan, np.nan),
    }

    overall_pass = all(bool(v) for v in checks.values())
    return checks, summary, overall_pass

analysis_for_validation = analysis_new if 'analysis_new' in globals() else extract_analysis_output(twoq_qst_result_new)
checks, summary, overall_pass = validate_twoq_qst_analysis(analysis_for_validation)

print('=== 2Q QST Validation Summary ===')
for k, v in checks.items():
    print(f'{_fmt_bool(v):4s} | {k}')
print('----------------------------------')
print('overall:', _fmt_bool(overall_pass))
print('assignment_avg_diag:', f"{summary['assignment_avg_diag']:.4f}")
print('assignment_row_sums:', np.round(summary['assignment_row_sums'], 4))
print('rho_trace_real:', f"{summary['rho_trace_real']:.8f}")
print('rho_min_eigenvalue:', f"{summary['rho_min_eigenvalue']:.3e}")
print('fidelity_to_target:', summary['fidelity_to_target'])
print('num_settings:', summary['num_settings'], 'shots[min,max]:', summary['shots_minmax'])
print('classifier diagnostics:', analysis_for_validation.get('classification_diagnostics', {}))


In [ ]:
A = np.asarray(analysis_new["assignment_matrix"], dtype=float)
print("col sums:", A.sum(axis=0))  # 이게 1에 가까워야 정상
print("row sums:", A.sum(axis=1))  # 필수 조건 아님


In [ ]:
# Optional repeatability check (set RUN_REPEAT = True to execute)
RUN_REPEAT = False
N_REPEAT = 3

if RUN_REPEAT:
    fids = []
    min_eigs = []
    diag_avgs = []
    pass_flags = []

    for rep in range(N_REPEAT):
        qst_rep = two_qubit_state_tomography.experiment_workflow(
            session=session,
            qpu=qpu,
            ctrl=ctrl,
            targ=targ,
            bus=bus,
            bus_frequency=bus_frequency,
            rip_amplitude=rip_amplitude,
            rip_length=rip_length,
            rip_phase=np.pi/2,
            readout_calibration_result=readout_cal_result_new,
            target_state='++',
            options=tomo_opts_new,
            temporary_parameters=temporary_parameters,
        ).run()

        an_rep = extract_analysis_output(qst_rep)
        checks_rep, summary_rep, ok_rep = validate_twoq_qst_analysis(an_rep)

        pass_flags.append(bool(ok_rep))
        diag_avgs.append(float(summary_rep['assignment_avg_diag']))
        min_eigs.append(float(summary_rep['rho_min_eigenvalue']))
        fid = summary_rep['fidelity_to_target']
        if fid is not None:
            fids.append(float(fid))

        print(f'Rep {rep+1}/{N_REPEAT}:', 'PASS' if ok_rep else 'FAIL',
              'diag=', f"{summary_rep['assignment_avg_diag']:.4f}",
              'min_eig=', f"{summary_rep['rho_min_eigenvalue']:.3e}",
              'fid=', summary_rep['fidelity_to_target'])

    print('\n=== Repeatability Summary ===')
    print('pass_rate:', f"{np.mean(pass_flags):.3f}")
    print('assignment_avg_diag: mean/std =', f"{np.mean(diag_avgs):.4f}", '/', f"{np.std(diag_avgs):.4f}")
    print('rho_min_eigenvalue: mean/std =', f"{np.mean(min_eigs):.3e}", '/', f"{np.std(min_eigs):.3e}")
    if len(fids) > 0:
        print('fidelity_to_target: mean/std =', f"{np.mean(fids):.4f}", '/', f"{np.std(fids):.4f}")
else:
    print('Set RUN_REPEAT = True to execute repeatability check.')


## Validation Mode Example (No RIP)

아래 셀은 `validation_mode=True`로 RIP 없이 `initial_state="++"`를 준비하고,
`target_state`는 자동으로 `++`로 맞춘 뒤 reconstructed density matrix를 ideal `++`와 비교합니다.


In [ ]:
bus = buses[0]
bus

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from experiments import two_qubit_state_tomography

# Reuse existing variables if present
# if "ctrl" not in globals() or "targ" not in globals():
#     ctrl = qubits[1]
#     targ = qubits[0]
# if "bus" not in globals():
#     bus = buses[0]
# if "temporary_parameters" not in globals():
#     temporary_parameters = {}

# Use existing readout calibration result when available
readout_cal_for_val = None
if "readout_cal_result_new" in globals():
    readout_cal_for_val = readout_cal_result_new

opts_val = two_qubit_state_tomography.experiment_workflow.options()
opts_val.do_analysis(True)
opts_val.do_readout_calibration(True)
opts_val.validation_mode(True)
opts_val.use_rip(True)              # validation_mode=True 이면 자동으로 RIP 미사용
opts_val.initial_state("++")
opts_val.enforce_target_match(True)

bus_frequency = (
    bus.parameters.resonance_frequency_bus + (bus.parameters.rip_detuning or 0.0)
    if bus.parameters.resonance_frequency_bus is not None
    else 6.5e9
)
rip_amplitude = float(getattr(bus.parameters, "rip_amplitude", 0.0) or 0.0)
rip_length = float(getattr(bus.parameters, "rip_length", 64e-9) or 64e-9)

wf_val = two_qubit_state_tomography.experiment_workflow(
    session=session,
    qpu=qpu,
    ctrl=ctrl,
    targ=targ,
    bus=bus,
    bus_frequency=bus_frequency,
    rip_amplitude=rip_amplitude,
    rip_length=rip_length,
    rip_phase=np.pi/2,
    readout_calibration_result=readout_cal_for_val,
    target_state=None,
    options=opts_val,
    temporary_parameters=temporary_parameters,
).run()




In [ ]:
analysis_val = extract_analysis_output(wf_val)
rho_hat = np.asarray(analysis_val["rho_hat_real"], dtype=float) + 1j*np.asarray(analysis_val["rho_hat_imag"], dtype=float)
rho_hat = (rho_hat + rho_hat.conj().T) / 2.0
rho_hat /= np.trace(rho_hat)

psi_pp = np.array([1,1,1,1], dtype=complex) / 2.0
rho_ideal = np.outer(psi_pp, psi_pp.conj())
fidelity_pp = float(np.real(np.conjugate(psi_pp) @ rho_hat @ psi_pp))

print("validation_mode:", unwrap_output(wf_val.output).get("validation_mode", "n/a"))
print("used_rip:", unwrap_output(wf_val.output).get("used_rip", "n/a"))
print("target_state_effective:", unwrap_output(wf_val.output).get("target_state_effective", "n/a"))
print("fidelity_to_++:", fidelity_pp)

labels = ["|00>","|01>","|10>","|11>"]
fig, axes = plt.subplots(1, 2, figsize=(10, 4.0))
im0 = axes[0].imshow(rho_ideal.real, vmin=-0.5, vmax=0.5, cmap="RdBu_r")
axes[0].set_title("Ideal Re[rho] for ++")
axes[0].set_xticks(range(4), labels, rotation=45)
axes[0].set_yticks(range(4), labels)

im1 = axes[1].imshow(rho_hat.real, vmin=-0.5, vmax=0.5, cmap="RdBu_r")
axes[1].set_title("Reconstructed Re[rho]")
axes[1].set_xticks(range(4), labels, rotation=45)
axes[1].set_yticks(range(4), labels)

fig.colorbar(im1, ax=axes.ravel().tolist(), fraction=0.035)
fig.tight_layout()
plt.show()

## Convergence Validation (Product State Suite)

이 섹션은 optimization convergence + statistical convergence를 함께 확인합니다.
statistical suite는 현재 pulse 지원 범위에 맞춰 `00, 01, 10, 11, ++`만 사용합니다.


In [ ]:
# Optional: run convergence suite workflow
RUN_CONVERGENCE_SUITE = True
CONVERGENCE_REPEATS = 3
CONVERGENCE_STATES = ("00", "01", "10", "11", "++")

if RUN_CONVERGENCE_SUITE:
    tomo_opts_conv = two_qubit_state_tomography.experiment_workflow.options()
    tomo_opts_conv.do_analysis(True)
    tomo_opts_conv.do_readout_calibration(False)
    tomo_opts_conv.do_convergence_validation(True)
    tomo_opts_conv.convergence_repeats_per_state(CONVERGENCE_REPEATS)
    tomo_opts_conv.convergence_suite_states(CONVERGENCE_STATES)
    tomo_opts_conv.convergence_do_plotting(True)

    twoq_qst_conv_result = two_qubit_state_tomography.experiment_workflow(
        session=session,
        qpu=qpu,
        ctrl=ctrl,
        targ=targ,
        bus=bus,
        bus_frequency=bus_frequency,
        rip_amplitude=rip_amplitude,
        rip_length=rip_length,
        rip_phase=np.pi / 2,
        readout_calibration_result=readout_cal_result_new,
        target_state="++",
        options=tomo_opts_conv,
        temporary_parameters=temporary_parameters,
    ).run()
    print("Convergence suite run finished.")
else:
    print("Set RUN_CONVERGENCE_SUITE = True to execute convergence suite.")


In [ ]:
def extract_convergence_report(workflow_result):
    out = unwrap_output(getattr(workflow_result, "output", workflow_result))
    if isinstance(out, dict) and "convergence_report" in out and not _contains_reference(out["convergence_report"]):
        return out["convergence_report"]

    report = {}
    stack = [workflow_result]
    seen = set()
    while stack:
        node = stack.pop()
        if node is None:
            continue
        nid = id(node)
        if nid in seen:
            continue
        seen.add(nid)

        for t in _iter_tasks(node):
            name = getattr(t, "name", "")
            if name == "resolve_convergence_suite_states":
                report["suite_states"] = tuple(unwrap_output(getattr(t, "output", None)) or ())
            elif name == "summarize_statistical_convergence":
                report["statistical_convergence"] = unwrap_output(getattr(t, "output", None))
            elif name == "extract_main_run_optimization_convergence":
                report["main_run_optimization_convergence"] = unwrap_output(getattr(t, "output", None))
            elif name == "collect_convergence_run_record":
                report.setdefault("raw_run_records", []).append(unwrap_output(getattr(t, "output", None)))
            stack.append(t)
            stack.append(getattr(t, "output", None))

        stack.append(getattr(node, "output", None))

    if "raw_run_records" in report:
        report["raw_run_records"] = [r for r in report["raw_run_records"] if isinstance(r, dict)]
    if "raw_run_records" in report and "suite_states" in report and report["suite_states"]:
        n_states = max(1, len(report["suite_states"]))
        report["repeats_per_state"] = int(len(report["raw_run_records"]) / n_states)
    return report

if "twoq_qst_conv_result" in globals():
    convergence_report = extract_convergence_report(twoq_qst_conv_result)
    print("Convergence report keys:", list(convergence_report.keys()))
    main_opt = convergence_report.get("main_run_optimization_convergence", {})
    if isinstance(main_opt, dict):
        print("main optimizer_success:", main_opt.get("optimizer_success"))
        print("main nll_per_shot:", main_opt.get("nll_per_shot"))
        print("main mae_counts:", main_opt.get("mae_counts"))
else:
    print("Run the convergence suite cell first.")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

if "convergence_report" not in globals():
    print("No convergence_report available.")
else:
    stat = convergence_report.get("statistical_convergence", {})
    per_state = stat.get("per_state", {}) if isinstance(stat, dict) else {}
    agg = stat.get("aggregate", {}) if isinstance(stat, dict) else {}

    print("=== Per-state statistical convergence ===")
    header = "state | runs | opt_success | fid_mean±std | fid_ci95 | nll_mean±std | min_eig_mean | min_eig_min"
    print(header)
    print("-" * len(header))
    for state in sorted(per_state.keys()):
        s = per_state[state]
        print(
            f"{state:>4s} | {s.get('num_runs', 0):>4d} | {s.get('optimizer_success_rate', 0.0):>10.3f} | "
            f"{s.get('fidelity_mean')} ± {s.get('fidelity_std')} | {s.get('fidelity_ci95')} | "
            f"{s.get('nll_mean')} ± {s.get('nll_std')} | {s.get('rho_min_eigenvalue_mean')} | {s.get('rho_min_eigenvalue_min')}"
        )

    print("\n=== Aggregate statistical convergence ===")
    for k in [
        "num_total_runs",
        "overall_optimizer_success_rate",
        "pooled_fidelity_mean",
        "pooled_fidelity_std",
        "pooled_fidelity_ci95",
        "worst_rho_min_eigenvalue",
    ]:
        print(f"{k}: {agg.get(k)}")

    states = sorted(per_state.keys())
    if states:
        means = np.array([per_state[s].get("fidelity_mean") if per_state[s].get("fidelity_mean") is not None else np.nan for s in states], dtype=float)
        errs = np.array([per_state[s].get("fidelity_ci95") if per_state[s].get("fidelity_ci95") is not None else np.nan for s in states], dtype=float)
        fig, ax = plt.subplots(figsize=(7.2, 3.8))
        x = np.arange(len(states))
        ax.errorbar(x, means, yerr=errs, fmt="o", capsize=4)
        ax.set_xticks(x, states)
        ax.set_ylim(0.0, 1.05)
        ax.set_ylabel("Fidelity")
        ax.set_title("Product-state suite fidelity mean ± 95% CI")
        ax.grid(alpha=0.3)
        plt.tight_layout()
        plt.show()


### Scope Note

이 convergence suite는 현재 Product cardinal subset(`00, 01, 10, 11, ++`)만 포함합니다.
Bell-state statistical convergence는 Bell preparation pulse가 추가된 뒤 확장합니다.


## Shot Sweep: Product-State Target Infidelity

`shots = 2^k (k=3..12)`에서 product state suite(`00, 01, 10, 11, ++`)의 target-state infidelity(`1 - fidelity_to_target`)를 측정합니다.


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

np.random.seed(20260222)

SHOT_LOG2_VALUES = list(range(3, 13))
SHOT_COUNTS = [2 ** k for k in SHOT_LOG2_VALUES]
PRODUCT_SUITE_STATES = ("00", "01", "10", "11", "++")
REPEATS_PER_POINT = 3
EPS = 1e-12
INFID_TOL = 1e-9

expected_total_runs = len(PRODUCT_SUITE_STATES) * len(SHOT_COUNTS) * REPEATS_PER_POINT

print("shot grid (log2):", SHOT_LOG2_VALUES)
print("shot counts:", SHOT_COUNTS)
print("states:", PRODUCT_SUITE_STATES)
print("repeats per point:", REPEATS_PER_POINT)
print("expected total runs:", expected_total_runs)


In [ ]:
if "readout_cal_result_new" not in globals():
    raise RuntimeError("readout_cal_result_new not found. Run readout calibration cell first.")
if "extract_analysis_output" not in globals():
    raise RuntimeError("extract_analysis_output function not found. Run QST setup cell first.")
if "temporary_parameters" not in globals():
    temporary_parameters = {}
if "ctrl" not in globals() or "targ" not in globals():
    ctrl = qubits[1]
    targ = qubits[0]
if "bus" not in globals():
    bus = buses[0]

bus_frequency = (
    bus.parameters.resonance_frequency_bus + (bus.parameters.rip_detuning or 0.0)
    if bus.parameters.resonance_frequency_bus is not None
    else 6.5e9
)
rip_amplitude = float(getattr(bus.parameters, "rip_amplitude", 0.0) or 0.0)
rip_length = float(getattr(bus.parameters, "rip_length", 64e-9) or 64e-9)


def _to_float_or_nan(value):
    try:
        v = float(value)
        return v if np.isfinite(v) else np.nan
    except Exception:
        return np.nan


sweep_rows = []
failed_runs = []
run_counter = 0

for state in PRODUCT_SUITE_STATES:
    for log2_shots, shots in zip(SHOT_LOG2_VALUES, SHOT_COUNTS):
        for repeat in range(1, REPEATS_PER_POINT + 1):
            run_counter += 1
            print(
                f"[{run_counter:03d}/{expected_total_runs}] "
                f"state={state}, shots={shots}, repeat={repeat}"
            )

            tomo_opts_sweep = two_qubit_state_tomography.experiment_workflow.options()
            tomo_opts_sweep.do_analysis(True)
            tomo_opts_sweep.do_readout_calibration(False)
            tomo_opts_sweep.validation_mode(True)
            tomo_opts_sweep.use_rip(False)
            tomo_opts_sweep.initial_state(state)
            tomo_opts_sweep.enforce_target_match(True)
            tomo_opts_sweep.count(int(shots))

            try:
                run_result = two_qubit_state_tomography.experiment_workflow(
                    session=session,
                    qpu=qpu,
                    ctrl=ctrl,
                    targ=targ,
                    bus=bus,
                    bus_frequency=bus_frequency,
                    rip_amplitude=rip_amplitude,
                    rip_length=rip_length,
                    rip_phase=np.pi / 2,
                    readout_calibration_result=readout_cal_result_new,
                    target_state=state,
                    options=tomo_opts_sweep,
                    temporary_parameters=temporary_parameters,
                ).run()
                analysis = extract_analysis_output(run_result)
            except Exception as exc:
                failed_runs.append(
                    {
                        "state": state,
                        "log2_shots": log2_shots,
                        "shots": shots,
                        "repeat": repeat,
                        "reason": repr(exc),
                    }
                )
                sweep_rows.append(
                    {
                        "state": state,
                        "log2_shots": log2_shots,
                        "shots": shots,
                        "repeat": repeat,
                        "fidelity": np.nan,
                        "infidelity": np.nan,
                        "log10_infidelity": np.nan,
                        "nll": np.nan,
                        "min_eig": np.nan,
                    }
                )
                print("  -> failed:", repr(exc))
                continue

            metrics = analysis.get("metrics", {}) if isinstance(analysis, dict) else {}
            fid_raw = metrics.get("fidelity_to_target")
            nll_raw = analysis.get("negative_log_likelihood") if isinstance(analysis, dict) else np.nan
            min_eig_raw = metrics.get("min_eigenvalue")

            fid = _to_float_or_nan(fid_raw)
            if np.isfinite(fid):
                infid = max(EPS, 1.0 - fid)
                log10_infid = float(np.log10(infid))
            else:
                infid = np.nan
                log10_infid = np.nan
                failed_runs.append(
                    {
                        "state": state,
                        "log2_shots": log2_shots,
                        "shots": shots,
                        "repeat": repeat,
                        "reason": f"invalid fidelity: {fid_raw!r}",
                    }
                )

            sweep_rows.append(
                {
                    "state": state,
                    "log2_shots": log2_shots,
                    "shots": shots,
                    "repeat": repeat,
                    "fidelity": fid,
                    "infidelity": infid,
                    "log10_infidelity": log10_infid,
                    "nll": _to_float_or_nan(nll_raw),
                    "min_eig": _to_float_or_nan(min_eig_raw),
                }
            )

shot_sweep_df = pd.DataFrame(
    sweep_rows,
    columns=[
        "state",
        "log2_shots",
        "shots",
        "repeat",
        "fidelity",
        "infidelity",
        "log10_infidelity",
        "nll",
        "min_eig",
    ],
)

print("Completed rows:", len(shot_sweep_df))
print("Failed/invalid runs:", len(failed_runs))
if failed_runs:
    display(pd.DataFrame(failed_runs).head(10))

display(shot_sweep_df.head())


In [ ]:
required_cols = [
    "state",
    "log2_shots",
    "shots",
    "repeat",
    "fidelity",
    "infidelity",
    "log10_infidelity",
    "nll",
    "min_eig",
]
assert list(shot_sweep_df.columns) == required_cols, "Unexpected dataframe schema."

counts_by_group = shot_sweep_df.groupby(["state", "log2_shots"]).size()
expected_index = pd.MultiIndex.from_product(
    [PRODUCT_SUITE_STATES, SHOT_LOG2_VALUES],
    names=["state", "log2_shots"],
)
missing_groups = expected_index.difference(counts_by_group.index)
bad_repeat_groups = counts_by_group[counts_by_group != REPEATS_PER_POINT]

range_violations = shot_sweep_df[
    shot_sweep_df["infidelity"].notna()
    & (
        (shot_sweep_df["infidelity"] < (EPS - 1e-15))
        | (shot_sweep_df["infidelity"] > (1.0 + INFID_TOL))
    )
]

print("=== Validation checks ===")
print("expected groups:", len(expected_index))
print("observed groups:", len(counts_by_group))
print("missing groups:", len(missing_groups))
print("groups with wrong repeat count:", len(bad_repeat_groups))
print("infidelity range violations:", len(range_violations))

counts_matrix = counts_by_group.unstack(level=1).reindex(PRODUCT_SUITE_STATES)
display(counts_matrix)

if len(missing_groups) > 0:
    print("Missing groups:", list(missing_groups)[:10])
if len(bad_repeat_groups) > 0:
    print("Bad repeat groups:")
    display(bad_repeat_groups)
if len(range_violations) > 0:
    print("Range violations (first 10):")
    display(range_violations.head(10))


In [ ]:
def _finite_stats(series):
    arr = pd.to_numeric(series, errors="coerce").to_numpy(dtype=float)
    arr = arr[np.isfinite(arr)]
    n = arr.size
    if n == 0:
        return {
            "n": 0,
            "mean": np.nan,
            "std": np.nan,
            "sem": np.nan,
            "ci95": np.nan,
        }
    mean = float(np.mean(arr))
    std = float(np.std(arr, ddof=1)) if n > 1 else 0.0
    sem = float(std / np.sqrt(n)) if n > 1 else 0.0
    ci95 = float(1.96 * sem) if n > 1 else 0.0
    return {
        "n": int(n),
        "mean": mean,
        "std": std,
        "sem": sem,
        "ci95": ci95,
    }


agg_rows = []
for (state, log2_shots), group in shot_sweep_df.groupby(["state", "log2_shots"], sort=True):
    inf_stats = _finite_stats(group["infidelity"])
    log_stats = _finite_stats(group["log10_infidelity"])
    agg_rows.append(
        {
            "state": state,
            "log2_shots": int(log2_shots),
            "shots": int(group["shots"].iloc[0]),
            "n_total": int(len(group)),
            "n_valid_infidelity": inf_stats["n"],
            "infid_mean": inf_stats["mean"],
            "infid_std": inf_stats["std"],
            "infid_sem": inf_stats["sem"],
            "infid_ci95": inf_stats["ci95"],
            "log10_infid_mean": log_stats["mean"],
            "log10_infid_std": log_stats["std"],
            "log10_infid_sem": log_stats["sem"],
            "log10_infid_ci95": log_stats["ci95"],
        }
    )

shot_sweep_stats = pd.DataFrame(agg_rows).sort_values(["state", "log2_shots"]).reset_index(drop=True)
display(shot_sweep_stats.head(15))


In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 5.5))

for state in PRODUCT_SUITE_STATES:
    state_df = shot_sweep_stats[shot_sweep_stats["state"] == state].sort_values("log2_shots")
    x = state_df["log2_shots"].to_numpy(dtype=float)

    y_infid = state_df["infid_mean"].to_numpy(dtype=float)
    e_infid = state_df["infid_ci95"].to_numpy(dtype=float)
    mask_infid = np.isfinite(y_infid)
    if np.any(mask_infid):
        axes[0].plot(x[mask_infid], y_infid[mask_infid], "o--", label=f"|{state}><{state}|")
        low = y_infid[mask_infid] - np.nan_to_num(e_infid[mask_infid], nan=0.0)
        high = y_infid[mask_infid] + np.nan_to_num(e_infid[mask_infid], nan=0.0)
        axes[0].fill_between(x[mask_infid], low, high, alpha=0.2)

    y_log = state_df["log10_infid_mean"].to_numpy(dtype=float)
    e_log = state_df["log10_infid_ci95"].to_numpy(dtype=float)
    mask_log = np.isfinite(y_log)
    if np.any(mask_log):
        axes[1].plot(x[mask_log], y_log[mask_log], "o--", label=f"|{state}><{state}|")
        low = y_log[mask_log] - np.nan_to_num(e_log[mask_log], nan=0.0)
        high = y_log[mask_log] + np.nan_to_num(e_log[mask_log], nan=0.0)
        axes[1].fill_between(x[mask_log], low, high, alpha=0.2)

axes[0].set_title("Infidelity vs Log(Number of shots)")
axes[0].set_xlabel("Log(Number of shots) = log2(shots)")
axes[0].set_ylabel("Infidelity")
axes[0].grid(alpha=0.3)
axes[0].legend(fontsize=9)

axes[1].set_title("Log10(Infidelity) vs Log(Number of shots)")
axes[1].set_xlabel("Log(Number of shots) = log2(shots)")
axes[1].set_ylabel("Log10(Infidelity)")
axes[1].grid(alpha=0.3)
axes[1].legend(fontsize=9)

fig.suptitle("2Q QST Product-State Suite Shot Sweep (k=3..12)", y=1.02, fontsize=14)
fig.tight_layout()
plt.show()


In [ ]:
last_log2 = max(SHOT_LOG2_VALUES)
last_shots = 2 ** last_log2

final_summary = (
    shot_sweep_stats[shot_sweep_stats["log2_shots"] == last_log2][
        [
            "state",
            "n_total",
            "n_valid_infidelity",
            "infid_mean",
            "infid_ci95",
            "log10_infid_mean",
            "log10_infid_ci95",
        ]
    ]
    .sort_values("state")
    .reset_index(drop=True)
)

print(f"Final-shot summary at log2(shots)={last_log2} (shots={last_shots})")
display(final_summary)
